# Read and clean a single file
---------
Nombre: **Julio Enrique Sanabria Quintana**

In [1]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'esp_bigdata'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [2]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [3]:
get_input_data?

Signature:
get_input_data(
    bucket='esp_bigdata',
    initial_directory='BigData',
    filename='datos-abiertos-agosto-2019.csv',
)
Docstring:
Read a csv file in a bucket of GCS, the file must use latin1 encoding and the separator is a semicolon (;)

Args:
    bucket (str, optional): Name of the bucket. Defaults to 'esp_bigdata'.
    initial_directory (str, optional): project directory. Defaults to 'BigData'.
    filename (str, optional):csv file to read. Defaults to 'datos-abiertos-agosto-2019.csv'.

Returns:
    pandas.dataframe: dataframe with the raw data
File:      ~/ESEIT_BigData/src/data/data_manipulator.py
Type:      function


In [4]:
raw_data = get_input_data(
    bucket= bucket,
    initial_directory=directory,
    filename = 'datos-abiertos-julio-2019.csv')
print(raw_data.info())
#raw_data.head()
#raw_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14560 entries, 0 to 14559
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   FECHA_INCIDENTE                    14560 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  14560 non-null  object
 2   CODIGO DE LOCALIDAD                14560 non-null  int64 
 3   LOCALIDAD                          14560 non-null  object
 4   EDAD                               14560 non-null  object
 5   UNIDAD                             14560 non-null  object
 6   GENERO                             14560 non-null  object
 7   RED                                14560 non-null  object
 8   TIPO_INCIDENTE                     14560 non-null  object
 9   PRIORIDAD                          14560 non-null  object
 10  MES                                14560 non-null  object
 11  CLASIFICACION FINAL                13035 non-null  object
dtypes: i

It seems that this file contains already the fields, but need to rename themhead

### Rename Columns

In [5]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True #sobreescribe el original
)
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [7]:
raw_data['LOCALIDAD'].value_counts(dropna=False) 
#dropna false para que me muetre los errores, "True" (defecto) no muetra errores

 Kennedy               1593
 Suba                  1548
Engativa               1521
 Bosa                  1122
 Ciudad Bolivar         994
Usaquen                 786
San Cristobal           776
 Rafael Uribe Uribe     741
Fontibon                723
 Puente Aranda          690
Usme                    636
Chapinero               599
Barrios Unidos          516
Los Martires            516
Santa Fe                489
Teusaquillo             464
Tunjuelito              462
 Antonio Nariño         292
 La Candelaria           91
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [8]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces ("strip"quita espacios en blanco)
raw_data['LOCALIDAD'].apply(     #apply para aplicar la función a toda la tabla
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               1593
Suba                  1548
Engativa              1521
Bosa                  1122
Ciudad Bolivar         994
Usaquen                786
San Cristobal          776
Rafael Uribe Uribe     741
Fontibon               723
Puente Aranda          690
Usme                   636
Chapinero              599
Barrios Unidos         516
Los Martires           516
Santa Fe               489
Teusaquillo            464
Tunjuelito             462
Antonio Nariño         292
La Candelaria           91
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [9]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [10]:
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [11]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     6621
MASCULINO    4190
FEMENINO     3749
Name: GENERO, dtype: int64

In [12]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    4261
Inconsciente/Paro Cardiorrespiratorio        1623
Dificultad Respiratoria                      1398
Enfermo                                      1131
Convulsiones                                 1011
Heridos                                       747
Trastorno mental                              709
Dolor torácico                                611
Lesiones personales                           547
Intento de suicidio                           534
Patología Gineco - obstétrica                 425
Síntomas gastrointestinales                   344
Accidente cerebro vascular                    323
Caída                                         318
Intoxicaciones                                141
Ideas de suicidio                             111
Violencia Sexual                               41
Muerte Natural                                 39
SIN_DATO                                       39
Incendio estructural                           29


In [17]:
col = 'CLASIFICACION_FINAL'
#raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Traslado                       6162
Cancelado                      3637
NaN                            1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [18]:
raw_data[col].fillna('SIN_DATO').value_counts()

Traslado                       6162
Cancelado                      3637
SIN_DATO                       1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

In [19]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,SIN_DATO
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [20]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_julio_2019.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [21]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 14560
Number of rows after cleaning: 13842


### Save the final table
use to_csv to save the table

In [22]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_julio_2019.csv', encoding='latin1', sep=';', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/ESEIT_BigData/data/processed/clean_llamadas_123_julio_2019.csv'